<a href="https://colab.research.google.com/github/thallyson1997/Adivinhe_o_Numero/blob/main/Detec%C3%A7%C3%A3o_de_objetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Introdução**

Neste projeto, utilizaremos a detecção de objetos com YOLO para identificar alimentos desperdiçados em bandejas do restaurante universitário ao fim das refeições. O objetivo é analisar o desperdício alimentar, contribuindo para iniciativas de redução de perdas e otimização do serviço.

Temos um conjunto de imagens das bandejas capturadas após as refeições, acompanhadas de arquivos `.txt` com as anotações das bounding boxes. As classes de alimentos que queremos identificar são:

- **Arroz**  
- **Farofa**  
- **Salada**  
- **Torta**  

O projeto visa treinar um modelo de detecção que identifique com precisão esses alimentos, facilitando a análise de padrões de desperdício e apoiando a tomada de decisões para ações corretivas.

### **2. Carregar Bibliotecas**

In [ ]:
# Carregar Bibliotecas
import os
import cv2
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import yaml
from glob import glob
import shutil
import random
import torch

# Instalar e importar a biblioteca ultralytics (YOLO)
!pip install ultralytics --quiet
from ultralytics import YOLO

# Verificar versão do PyTorch e dispositivo disponível
print(f'PyTorch version: {torch.__version__}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Usando dispositivo: {device}')

PyTorch version: 2.5.1+cu124
Usando dispositivo: cpu


### **3. Carregar e Explorar Dados**

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Definir caminho para o diretório com imagens e anotações
dataset_path = '/content/drive/MyDrive/PDI/Alimentos'

# Ajustar caminhos para encontrar imagens e anotações
image_files = sorted(glob(os.path.join(dataset_path, '**', '*.JPG'), recursive=True))
label_files = {os.path.basename(f).replace('.txt', ''): f for f in
                glob(os.path.join(dataset_path, '**', '*.txt'), recursive=True)}

print(f'Total de imagens encontradas: {len(image_files)}')
print(f'Total de anotações encontradas: {len(label_files)}')

# Criar dataset com colunas para cada classe
dataset_info = []
for img_path in image_files:
    image_name = os.path.basename(img_path).replace('.JPG', '')
    label_path = label_files.get(image_name)

    classes = []
    if label_path:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            classes = sorted(set(int(line.split()[0]) for line in lines))

    dataset_info.append({
        'imagem': os.path.basename(img_path),
        'tem_anotacao': bool(label_path),
        'classe_0': 1 if 0 in classes else 0,
        'classe_1': 1 if 1 in classes else 0,
        'classe_2': 1 if 2 in classes else 0,
        'classe_3': 1 if 3 in classes else 0
    })

# Converter para DataFrame e exibir amostra
dataset_df = pd.DataFrame(dataset_info)
print(dataset_df.head())

# Salvar dataset em CSV
dataset_df.to_csv(os.path.join(dataset_path, 'dataset_info.csv'), index=False)
print('Arquivo dataset_info.csv salvo com sucesso.')

Total de imagens encontradas: 68
Total de anotações encontradas: 58
          imagem  tem_anotacao  classe_0  classe_1  classe_2  classe_3
0   foto (1).JPG          True         1         1         1         1
1  foto (10).JPG          True         1         0         0         0
2  foto (11).JPG          True         1         1         1         0
3  foto (12).JPG          True         1         0         1         0
4  foto (13).JPG          True         1         1         1         0
Arquivo dataset_info.csv salvo com sucesso.


In [ ]:
dataset_df.head(20)

,imagem,tem_anotacao,classe_0,classe_1,classe_2,classe_3
0,foto (1).JPG,True,1,1,1,1
1,foto (10).JPG,True,1,0,0,0
2,foto (11).JPG,True,1,1,1,0
3,foto (12).JPG,True,1,0,1,0
4,foto (13).JPG,True,1,1,1,0
5,foto (14).JPG,True,1,1,1,0
6,foto (15).JPG,True,0,1,0,0
7,foto (16).JPG,True,0,1,0,0
8,foto (17).JPG,True,0,1,0,0
9,foto (18).JPG,True,1,1,1,0



### **4. Preparar os Dados**

In [ ]:
# Definir diretórios para treinamento e teste
train_dir = os.path.join(dataset_path, 'train')
val_dir = os.path.join(dataset_path, 'val')

# Criar diretórios, se não existirem
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Função para dividir dados em treino e teste
def split_data(image_files, train_ratio=0.8):
    random.shuffle(image_files)
    train_size = int(len(image_files) * train_ratio)
    train_images = image_files[:train_size]
    val_images = image_files[train_size:]
    return train_images, val_images

# Dividir os dados
train_images, val_images = split_data(image_files)

# Mover as imagens para os diretórios apropriados
def move_images(image_files, target_dir):
    for img_path in image_files:
        shutil.copy(img_path, target_dir)
        label_path = label_files.get(os.path.basename(img_path).replace('.JPG', ''))
        if label_path:
            shutil.copy(label_path, target_dir)

move_images(train_images, train_dir)
move_images(val_images, val_dir)

In [ ]:
# Definir o arquivo YAML para treinamento
data_yaml = {
    'train': train_dir,
    'val': val_dir,
    'nc': 4,  # número de classes
    'names': ['Arroz', 'Farofa', 'Salada', 'Torta']
}

# Salvar arquivo YAML
yaml_path = os.path.join(dataset_path, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f'Arquivo de configuração salvo em {yaml_path}')

Arquivo de configuração salvo em /content/drive/MyDrive/PDI/Alimentos/data.yaml



### **5. Configurar e Treinar o Modelo YOLO**

In [ ]:
# Carregar modelo YOLOv8 (pode ser uma versão específica como "yolov8n" ou "yolov8s")
model = YOLO("yolov8n.pt")  # "yolov8n.pt" é uma versão pequena, você pode usar "yolov8s.pt" ou outra, conforme a necessidade.

# Mostrar as configurações do modelo
print(model)

100%|██████████| 6.25M/6.25M [00:00<00:00, 281MB/s]

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
# Definir configurações de treinamento
train_args = {
    'data': yaml_path,  # Caminho do arquivo YAML
    'epochs': 50,  # Número de épocas para treinamento
    'batch': 16,  # Tamanho do lote (batch size)
    'imgsz': 640,  # Tamanho da imagem para o treinamento
    'device': device,  # Dispositivo (GPU ou CPU)
    'project': '/content/drive/MyDrive/PDI/Alimentos',  # Diretório onde o projeto será salvo
    'name': 'treinamento_yolo',  # Nome do experimento
    'exist_ok': True,  # Se True, sobrescreve o diretório do projeto
}

# Iniciar o treinamento
model.train(**train_args)

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/PDI/Alimentos/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=/content/drive/MyDrive/PDI/Alimentos, name=treinamento_yolo, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

100%|██████████| 755k/755k [00:00<00:00, 89.8MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/PDI/Alimentos/train... 45 images, 9 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<00:00, 134.42it/s]


train: New cache created: /content/drive/MyDrive/PDI/Alimentos/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all         14         28       0.55      0.617      0.586      0.363



50 epochs completed in 0.657 hours.
Optimizer stripped from /content/drive/MyDrive/PDI/Alimentos/treinamento_yolo/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/PDI/Alimentos/treinamento_yolo/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/PDI/Alimentos/treinamento_yolo/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.25s/it]


                   all         14         28       0.64      0.634      0.627      0.369
                 Arroz         12         14      0.645      0.786      0.751      0.582
                Farofa          4          6      0.568        0.5       0.42      0.292
                Salada          3          4      0.368       0.75      0.621      0.203
                 Torta          4          4      0.981        0.5      0.717        0.4
Speed: 2.2ms preprocess, 211.2ms inference, 0.0ms loss, 6.6ms postprocess per image
Results saved to /content/drive/MyDrive/PDI/Alimentos/treinamento_yolo


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aac40802e50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0


### **6. Avaliar o Modelo**


### **7. Testar com Novas Imagens**


### **8. Conclusão e Próximos Passos**